In [1]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import string
import spacy
from spacy import displacy

In [74]:
EM_sentiment = pd.read_csv('Elon_musk.csv', encoding='ISO-8859-1',index_col=0)

In [75]:
EM_sentiment

,Text
1,@kunalb11 Im an alien
2,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
3,@joerogan @Spotify Great interview!
4,@gtera27 Doge is underestimated
5,@teslacn Congratulations Tesla China for amazi...
...,...
1995,"@flcnhvy True, it sounds so surreal, but the n..."
1996,@PPathole Make sure to read ur terms &amp; con...
1997,@TeslaGong @PPathole Samwise Gamgee
1998,@PPathole Altho Dumb and Dumber is <U+0001F525...


In [76]:
len(EM_sentiment)

1999

In [77]:
EM_sentiment['Text'] = EM_sentiment['Text'].apply(lambda x: re.sub('@[^\s]+', '', x))

EM_sentiment['Text'] = EM_sentiment['Text'].apply(lambda x: re.sub(r'http\S+', '', x))

EM_sentiment['Text'] = EM_sentiment['Text'].apply(lambda x: x.lower())

EM_sentiment['Text'] = EM_sentiment['Text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

EM_sentiment['Text'] = EM_sentiment['Text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

In [78]:
EM_sentiment

,Text
1,im an alien
2,ray tracing on cyberpunk with hdr is nextleve...
3,great interview
4,doge is underestimated
5,congratulations tesla china for amazing execu...
...,...
1995,true it sounds so surreal but the negative pr...
1996,make sure to read ur terms amp conditions bef...
1997,samwise gamgee
1998,altho dumb and dumber is u0001f525u0001f525


In [71]:
lemma =WordNetLemmatizer()

In [ ]:
corpus = []

for text in EM_sentiment['Text']:
    txt = re.sub('[^a-zA-Z]',' ', text)
    txt = txt.lower()
    txt = txt.split()

    txt = [lemma.lemmatize(i) for i in txt if not i in stopwords.words('english')]
    txt = ' '.join(txt)
    corpus.append(txt)

In [80]:
corpus

[1        im an alien  ray tracing on cyberpunk with hd...
 2        im an alien  ray tracing on cyberpunk with hd...
 3        im an alien  ray tracing on cyberpunk with hd...
 4        im an alien  ray tracing on cyberpunk with hd...
 5        im an alien  ray tracing on cyberpunk with hd...
                               ...                        
 1995     im an alien  ray tracing on cyberpunk with hd...
 1996     im an alien  ray tracing on cyberpunk with hd...
 1997     im an alien  ray tracing on cyberpunk with hd...
 1998     im an alien  ray tracing on cyberpunk with hd...
 1999     im an alien  ray tracing on cyberpunk with hd...
 Name: Text, Length: 1999, dtype: object]

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [21]:
cv = CountVectorizer()
x = cv.fit_transform(corpus)

In [24]:
X = pd.DataFrame(x.toarray(), columns=cv.get_feature_names_out())
X

,ab,aber,able,abo,aboard,abort,absence,absolute,absolutely,absorb,...,young,youre,youve,yup,zealand,zenit,zero,zip,zon,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
EM_sentiment['label'] = EM_sentiment['sentiments'].map({'positive': 0, 'negative': 1, 'neutral': 2})

KeyError: 'sentiments'

In [25]:
import numpy as np
sentiments = ["positive", "negative", "neutral"]
labels = np.array(sentiments)

In [32]:
for i, label in zip(range(len(labels)), labels):
    if label == 0:
        X.loc[i, 'label'] = 'negative'
    elif label == 1:
        X.loc[i, 'label'] = 'positive'
    else:
        X.loc[i, 'label'] = 'neutral'

In [34]:
X

,ab,aber,able,abo,aboard,abort,absence,absolute,absolutely,absorb,...,youre,youve,yup,zealand,zenit,zero,zip,zon,zone,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neutral
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neutral
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neutral
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [17]:
for i in corpus:
    words = word_tokenize(i)
    for i in words:
        if i not in stopwords.words('english'):
            print(lemma.lemmatize(i))

text
im
alien
ray
tracing
cyberpunk
hdr
nextleve
great
interview
doge
underestimated
congratulation
tesla
china
amazing
execu
true
sound
surreal
negative
pr
make
sure
read
ur
term
amp
condition
bef
samwise
gamgee
altho
dumb
dumber
u
f
u
f
progress
update
august
row
x
column


In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [19]:
cv = CountVectorizer()

x = cv.fit_transform(corpus)
x.toarray()

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [20]:
cv.get_feature_names_out()

array(['alien', 'altho', 'amazing', 'amp', 'an', 'and', 'august', 'bef',
       'but', 'china', 'columns', 'conditions', 'congratulations',
       'cyberpunk', 'doge', 'dumb', 'dumber', 'execu', 'for', 'gamgee',
       'great', 'hdr', 'im', 'interview', 'is', 'it', 'make', 'negative',
       'nextleve', 'on', 'pr', 'progress', 'ray', 'read', 'rows',
       'samwise', 'so', 'sounds', 'sure', 'surreal', 'terms', 'tesla',
       'text', 'the', 'to', 'tracing', 'true', 'underestimated', 'update',
       'ur', 'with'], dtype=object)

In [21]:
cv.vocabulary_

{'text': 42,
 'im': 22,
 'an': 4,
 'alien': 0,
 'ray': 32,
 'tracing': 45,
 'on': 29,
 'cyberpunk': 13,
 'with': 50,
 'hdr': 21,
 'is': 24,
 'nextleve': 28,
 'great': 20,
 'interview': 23,
 'doge': 14,
 'underestimated': 47,
 'congratulations': 12,
 'tesla': 41,
 'china': 9,
 'for': 18,
 'amazing': 2,
 'execu': 17,
 'true': 46,
 'it': 25,
 'sounds': 37,
 'so': 36,
 'surreal': 39,
 'but': 8,
 'the': 43,
 'negative': 27,
 'pr': 30,
 'make': 26,
 'sure': 38,
 'to': 44,
 'read': 33,
 'ur': 49,
 'terms': 40,
 'amp': 3,
 'conditions': 11,
 'bef': 7,
 'samwise': 35,
 'gamgee': 19,
 'altho': 1,
 'dumb': 15,
 'and': 5,
 'dumber': 16,
 'progress': 31,
 'update': 48,
 'august': 6,
 'rows': 34,
 'columns': 10}

In [22]:
corpus[0]

'                                                   text                                             im an alien        ray tracing on cyberpunk with hdr is nextleve                                            great interview                                  doge is underestimated        congratulations tesla china for amazing execu                                                                   true it sounds so surreal but the negative pr           make sure to read ur terms amp conditions bef                                             samwise gamgee             altho dumb and dumber is u    f   u    f                                  progress update august           rows x   columns '

In [23]:
df = pd.DataFrame(x.toarray(), columns=cv.get_feature_names_out())
df

,alien,altho,amazing,amp,an,and,august,bef,but,china,...,tesla,text,the,to,tracing,true,underestimated,update,ur,with
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [24]:
tf = TfidfVectorizer()

x = tf.fit_transform(corpus)

df = pd.DataFrame(x.toarray(), columns=tf.get_feature_names_out())
df

,alien,altho,amazing,amp,an,and,august,bef,but,china,...,tesla,text,the,to,tracing,true,underestimated,update,ur,with
0,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,...,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189


In [42]:
df['sentiment'] = 'positive'

In [44]:
df['sentiment'] = 'negative'
df['sentiment'] = 'neutral'

In [45]:
df['labels'] = df['sentiment'].map({'positive': 0, 'negative': 1, 'neutral': 2})

In [48]:
df['labels']

0    2
Name: labels, dtype: int64

In [49]:
df

,alien,altho,amazing,amp,an,and,august,bef,but,china,...,the,to,tracing,true,underestimated,update,ur,with,sentiment,labels
0,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,...,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,0.130189,neutral,2


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=1)

ValueError: Found input variables with inconsistent numbers of samples: [1, 3]

In [ ]:
model = LinearSVC(random_state=0)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)